In [15]:
import os
path = os.getcwd()
path

'/Users/sophiacofone/school_local/svi_worm_anno'

In [2]:
from skimage import data
import napari

In [3]:
import skimage.measure as meas

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import tifffile

In [6]:
from pycromanager import Dataset

In [7]:
fname = 'StripA_cent1G_5h_jub39R_5h_6h_devA1_1_projected.tiff'

In [8]:
image_data = tifffile.imread(fname)
shape = image_data.shape
print(shape)

(16, 3, 2048, 2048)


In [12]:
import napari
import numpy as np
from vispy.color import Colormap
import tifffile as tiff

In [13]:
class mySet:
    def __init__(self,path,name,data):
        self.path = path
        self.name = name
        self.data = data
        self.current_w = None
        self.num_w = data.shape[0]
        self.q = np.arange(self.num_w).tolist()
        
    def get_next_w(self):
        self.current_w = self.q.pop()
        print(self.current_w)
        return self.data[self.current_w]
    
    def save_annot(self,annot_stack):
        tiff.imsave(os.path.join(self.path,self.name+'_{}.tiff'.format(self.current_w)),annot_stack)
        
        
def open_worm(viewer,wstack):
    viewer.add_image(
        wstack,
        name=[f"worm, Channel Brightfield"],
        colormap=["gray"],
    )
    
def save_open_next(viewer,myS):
    if myS.current_w is None:
        open_worm(viewer,myS.get_next_w())
    else:
        try:
            names = [l.name for l in viewer.layers]
            worm_mask_layer = viewer.layers[[i for i,j in enumerate(names) if j == 'worm_mask'][0]].data
        
            stacked_images = np.dstack((worm_mask_layer)).transpose(1, 0, 1)
            myS.save_annot(stacked_images)

            viewer.layers.clear()
        except IndexError:
            print('annotation layers not added yet')
            return
        
        try:
            open_worm(viewer,myS.get_next_w())
        except IndexError:
            print('finished anotating set')
            


previous_layer = None  # Variable to store the previous layer

def add_layer(viewer, color):
    global previous_layer

    # Deactivate the previous layer
    if previous_layer is not None:
        previous_layer.visible = False

    # Create a new labels layer
    labels = np.zeros((2048, 2048), dtype=np.uint16)
    anot_layer = viewer.add_labels(labels, name=color, seed=0.2)
    anot_layer.brush_size = 25
    anot_layer.mode = 'paint'

    # Update the previous layer variable
    previous_layer = anot_layer
        


In [43]:
myS = mySet(path,'test_set_worm',image_data)

viewer = napari.Viewer()

viewer.bind_key('w', lambda event: add_layer(viewer, 'worm_mask'))
viewer.bind_key('c', lambda event: save_open_next(viewer,myS))


15
annotation layers not added yet
annotation layers not added yet
annotation layers not added yet
